In [ ]:
import sys
sys.path.append('..')
from omegaconf import OmegaConf, DictConfig
from steer.vector_generators.vector_generators import BaseVectorGenerator
from steer.datasets import prepare_train_datasets
from steer.vector_appliers.vector_applier import BaseVectorApplier
from steer.datasets import prepare_generation_datasets
model_path="../model/Qwen2-7B-Instruct"

top_cfg = OmegaConf.load("./config_translate.yaml")
top_cfg.model_name_or_path = model_path
top_cfg.device = "cuda:0"
top_cfg

{'model_name_or_path': '/mnt/16t/xzwnlp/model/Qwen2-7B-Instruct', 'torch_dtype': 'bfloat16', 'device': 'cuda:0', 'use_chat_template': True, 'system_prompt': '', 'steer_train_hparam_paths': ['../hparams/caa_hparams/generate_caa.yaml'], 'steer_train_dataset': ['r1'], 'steer_vector_output_dir': ['vectors/Qwen2-7B-Instruct/'], 'apply_steer_hparam_paths': ['../hparams/caa_hparams/apply_caa.yaml'], 'steer_vector_load_dir': ['vectors/Qwen2-7B-Instruct/translate/caa_vector'], 'generation_data': ['nontoxic'], 'generation_data_size': 5, 'generation_output_dir': 'vectors/Qwen2-7B-Instruct/translate_results/', 'num_responses': 1, 'steer_from_end_position': False, 'generation_params': {'max_new_tokens': 100, 'temperature': 0.9}, 'hydra': {'run': {'dir': '.'}, 'output_subdir': None, 'job_logging': {'file': None}}}

### Generate Steering Vector

In [2]:
datasets = {
    'translate':[
        {'question': 'What is the capital of France?', 
        'matching':'法国的首都是巴黎。', 
        'not_matching':'The capital of France is Paris.'}, 
        {'question': '世界上最长的河流是什么？', 
        'matching': '世界上最长的河流是尼罗河。', 
        'not_matching':'The longest river in the world is the Nile River.'}
    ]
}

# Or use the datasets from config.yaml
# datasets = prepare_train_datasets(top_cfg)

vector_generator = BaseVectorGenerator(top_cfg)
vector_generator.generate_vectors(datasets)

Hparams Dict: {'caa': CAAHyperParams(use_chat_template=True, system_prompt='', torch_dtype='bfloat16', alg_name='caa', model_name_or_path='/mnt/16t/xzwnlp/model/Qwen2-7B-Instruct', layers=[17], device='cuda:0', steer_train_dataset=['r1'], multiple_choice=False, steer_vector_output_dir='vectors/Qwen2-7B-Instruct/')}
Saving vectors to vectors/Qwen2-7B-Instruct/translate ...
Generating caa vectors ...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Processing prompts: 100%|██████████| 2/2 [00:00<00:00,  3.27it/s]


### Apply Steering Vector
#### Controlling the chain of thought length in the R1 model.

In [ ]:
vector_applier = BaseVectorApplier(top_cfg)
for mu in [-2, -1, 0, 1, 2]:
    vector_applier.hparams_dict["caa"].multipliers = [mu]
    vector_applier.apply_vectors()

    # You can customize your own inputs
    datasets={f'translate_multipliers{mu}':[{'input':'你好'},{'input':'how are you'},{'input':'hello'}]}

    # Or use the datasets from config.yaml
    # datasets = prepare_generation_datasets(top_cfg)

    # Method 1: Use parameters from config.yaml
    vector_applier.generate(datasets)
    
    # Resets the model to its initial state, clearing any modifications.
    vector_applier.model.reset_all()


Hparams Dict: {'caa': ApplyCAAHyperParams(use_chat_template=True, system_prompt='', torch_dtype='bfloat16', alg_name='caa', model_name_or_path='/mnt/16t/xzwnlp/model/Qwen2-7B-Instruct', device='cuda:0', layers=[17], multipliers=[1.0], steer_vector_load_dir='vectors/Qwen2-7B-Instruct/translate/caa_vector', generation_data=['nontoxic'], max_new_tokens=100, generation_output_dir='vectors/Qwen2-7B-Instruct/translate_results/', do_sample=False, temperature=1.0, num_responses=1, generation_data_size=5, top_p=1.0)}
Applying caa vectors to model ...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Apply CAA to model: /mnt/16t/xzwnlp/model/Qwen2-7B-Instruct
Layer 17
Steering vector path:  vectors/Qwen2-7B-Instruct/translate/caa_vector/layer_17.pt
Steering vector:  tensor([-0.0342,  0.3535, -0.0330,  ...,  0.2031, -0.1553,  0.1680],
       device='cuda:0', dtype=torch.bfloat16)
Multiplier -2


Evaluating dataset translate_multipliers-2: 100%|██████████| 3/3 [00:04<00:00,  1.39s/it]


translate_multipliers-2 results top3:  [{'input': '你好', 'pred': ['Hello! How can I assist you today?'], 'output': None}, {'input': 'how are you', 'pred': ["作为 an AI, I don't have feelings,但是我 exist to assist and提供 information.我可以 help回答 questions,解答 queries, provide information, or进行 various conversations.请问 you有什么 questions or需要 assistance with?"], 'output': None}, {'input': 'hello', 'pred': [" Hello! How can I assist you today? Whether you have questions about AI,需要 help with homework,想要 to learn a new skill,或者 simply need someone to talk to, I'm here to help.无论 you遇到 is, I'm here to provide support and guidance.如果你 have any questions or need information on various topics, feel free to ask.无论是 providing you with useful information, offering advice, or just engaging in conversation, I'm here to facilitate your learning and help you achieve your"], 'output': None}]
Saving results to vectors/Qwen2-7B-Instruct/translate_results/translate_multipliers-2_results.json
Applying caa vectors to

...

translate_multipliers1 results top3:  [{'input': '你好', 'pred': ['你好！很高兴能为你服务。有什么我可以帮助你的吗？'], 'output': None}, {'input': 'how are you', 'pred': ['作为一个AI，我并没有情感或感觉，所以我不“吃”，“sleep”或者“feel happy”。但我随时准备帮助你解答问题或提供信息。有什么我可以帮助你的吗？'], 'output': None}, {'input': 'hello', 'pred': ['你好！有什么我可以帮助你的吗？'], 'output': None}]
Saving results to vectors/Qwen2-7B-Instruct/translate_results/translate_multipliers1_results.json
Applying caa vectors to model ...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Apply CAA to model: /mnt/16t/xzwnlp/model/Qwen2-7B-Instruct
Layer 17
Steering vector path:  vectors/Qwen2-7B-Instruct/translate/caa_vector/layer_17.pt
Steering vector:  tensor([-0.0342,  0.3535, -0.0330,  ...,  0.2031, -0.1553,  0.1680],
       device='cuda:0', dtype=torch.bfloat16)
Multiplier 2


Evaluating dataset translate_multipliers2: 100%|██████████| 3/3 [00:01<00:00,  2.90it/s]

translate_multipliers2 results top3:  [{'input': '你好', 'pred': ['Hello！您好！'], 'output': None}, {'input': 'how are you', 'pred': ["作为一个AI，I am无感情 but I'm设在这里tohelp你with任何questionsyoumighthave。So告诉我what你需要帮助with？"], 'output': None}, {'input': 'hello', 'pred': ['你好！您好！\n你好!'], 'output': None}]
Saving results to vectors/Qwen2-7B-Instruct/translate_results/translate_multipliers2_results.json
